In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import tensorflow as tf

In [7]:
from platform import python_version

print(python_version())


3.11.9


In [8]:
tf.__version__

'2.16.2'

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# GAP2D with Normalization and Dropout

In [ ]:
import tensorflow.keras.applications as pmodels
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
strategy = tf.distribute.MirroredStrategy()

def test_base_models():
    images = image_dataset_from_directory("../data/kaggle/input/stanford-dogs-dataset-traintest/cropped/train", 
                                      seed=1234, subset="both", validation_split=0.2, batch_size=64, 
                                      labels="inferred", label_mode = "categorical")
    test_images = image_dataset_from_directory("../data/kaggle/input/stanford-dogs-dataset-traintest/cropped/test", 
                                      seed=1234, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    base_models = [pmodels.efficientnet_v2.EfficientNetV2S]
    preprocess_func = [do_nothing]
    histories = []
    scores = []
    models = []
    for index, base_model in enumerate(base_models):
        model = build_model(base_model)
        images_train = images[0].map(lambda x, y: (preprocess_func[index](x), y))
        images_val = images[1].map(lambda x, y: (preprocess_func[index](x), y))
        images_test = test_images.map(lambda x, y: (preprocess_func[index](x), y))
        history = train_model(model, images_train, images_val)
        score = evaluate_model(model, images_test)
        histories.append(history)
        scores.append(score)
        models.append(model)
        model.save('../snoop_dog/models/effnetv2s_finetuned_valsplit01.keras')
    return histories, scores, models, base_models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    gap_layer = layers.GlobalAveragePooling2D()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(120, activation='softmax')

    model = models.Sequential([
      model,
      gap_layer,
      dense_layer,
      prediction_layer
    ])

    return model

def build_model(base_model):

    model = load_model(base_model)
    model = set_nontrainable_layers(model)
    model = add_last_layers(model)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
    return model

def load_model(base_model):
    model = base_model(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    return model

def set_nontrainable_layers(model):
    model.trainable = False
    return model

def train_model(model, images_train, images_val):
    model.summary()
    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    history = model.fit(images_train,
              epochs=10,
              batch_size=64,
              callbacks=es,
              validation_data=images_val,
              verbose=1)
    return history

def evaluate_model(model, images_test):
    return model.evaluate(images_test)

def do_nothing(object):
    return object

histories, scores, models, base_models = test_base_models()